In [1]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

We shall use the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

To know more about ImageDataGenerator class, visit https://keras.io/preprocessing/image/#imagedatagenerator-class

In [2]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    'train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    'test',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary')

Found 392 images belonging to 2 classes.
Found 258 images belonging to 2 classes.


Our model will have 3 Convolution2D layers. You can increse or decrease as per your needs.

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),     
     
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2), 
    
    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       2

We shall use Adam optimizer with a learning rate of 0.001 (hyperparameter). We choose 'binary_crossentropy' loss as our model is a binary calssifier (i.e, we have only 2 classes)

In [5]:
# model.compile(loss='binary_crossentropy',
#              optimizer=tf.keras.optimizers.Adam(0.001),
#              metrics=['accuracy'])

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
# opt = tf.keras.optimizers.SGD(learning_rate=0.00001)

model.compile(loss='binary_crossentropy',
             optimizer = opt,
             metrics=['accuracy'])

At the end of each epoch we can check if the model has reached the required accuracy and terminate the training.

In [6]:
# DESIRED_ACCURACY = 0.85

# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if((logs.get('acc')>DESIRED_ACCURACY) and (logs.get('val_acc')>DESIRED_ACCURACY )):
#       print("\nReached 85% accuracy so cancelling training!")
#       self.model.stop_training = True

# callbacks = myCallback()

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

To know more about fit_generator visit https://keras.io/models/model/#fit_generator

In [7]:
my_callback = MyThresholdCallback(threshold=0.85)

model.fit_generator(
    train_datagenerator,
    epochs=100,
    validation_data = test_datagenerator,
    callbacks = [my_callback]
    )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
13/13 [==============================] - 4s 273ms/step - loss: 0.6937 - accuracy: 0.5332 - val_loss: 0.6705 - val_accuracy: 0.6085
Epoch 2/100
13/13 [==============================] - 3s 239ms/step - loss: 0.6860 - accuracy: 0.5434 - val_loss: 0.7106 - val_accuracy: 0.3953
Epoch 3/100
13/13 [==============================] - 3s 255ms/step - loss: 0.6655 - accuracy: 0.5638 - val_loss: 0.6926 - val_accuracy: 0.4690
Epoch 4/100
13/13 [==============================] - 3s 251ms/step - loss: 0.6468 - accuracy: 0.6352 - val_loss: 0.6281 - val_accuracy: 0.7171
Epoch 5/100
13/13 [==============================] - 3s 255ms/step - loss: 0.6190 - accuracy: 0.6556 - val_loss: 0.6289 - val_accuracy: 0.6860
Epoch 6/100
13/13 [==============================] - 3s 262ms/step - loss: 0.5725 - accuracy: 0.6939 - val_loss: 0.6252 - val_accuracy: 0.6473
Epoch 7/100
13/13 [==============================] - 3s 264ms/step

13/13 [==============================] - 3s 259ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.5794 - val_accuracy: 0.8178
Epoch 57/100
13/13 [==============================] - 3s 262ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.6799 - val_accuracy: 0.8101
Epoch 58/100
13/13 [==============================] - 3s 236ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.6532 - val_accuracy: 0.7984
Epoch 59/100
13/13 [==============================] - 3s 251ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6352 - val_accuracy: 0.8101
Epoch 60/100
13/13 [==============================] - 3s 242ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6525 - val_accuracy: 0.8062
Epoch 61/100
13/13 [==============================] - 3s 254ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6580 - val_accuracy: 0.8101
Epoch 62/100
13/13 [==============================] - 3s 264ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6278 - val_accuracy: 0.8140
Epoch

In [8]:
model.save('Pizza.h5')